In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_columns', None)
# #pd.options.display.float_format = '{:.2f}'.format

In [2]:
# The path to our CSV files
constructioncsv = "Issued_Construction_Permits.csv"

# Read our csvfiles data into pandas
constructionPermit_df = pd.read_csv(constructioncsv)

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,23,24,25,27,28,29,30,31,32,33,34,35,36,38,53,54,56,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
constructionpermit2_df = constructionPermit_df.loc[constructionPermit_df["CalendarYearIssued"]>=1996,:]

In [4]:
residential_df = constructionpermit2_df.loc[constructionpermit2_df["PermitClassMapped"]=="Residential",:]
commercial_df = constructionpermit2_df.loc[constructionpermit2_df["PermitClassMapped"]=="Commercial",:]

In [5]:
comm_residential_df1 = commercial_df.loc[commercial_df["PermitClass"]=="C- 101 Single Family Houses",:]
overall_residential_df = residential_df.append(comm_residential_df1, ignore_index=True)

In [6]:
overall_residential_BP = overall_residential_df.loc[overall_residential_df["PermitType"]=="BP",:]
overall_residential_BP["TotalJobValuation"] = overall_residential_BP["TotalJobValuation"].str.replace('$','')
overall_residential_BP["TotalJobValuation"] = overall_residential_BP["TotalJobValuation"].str.replace(',','')
overall_residential_BP["TotalValuationRemodel"] = overall_residential_BP["TotalValuationRemodel"].str.replace('$','')
overall_residential_BP["TotalValuationRemodel"] = overall_residential_BP["TotalValuationRemodel"].str.replace(',','')
overall_residential_BP["TotalJobValuation"].fillna(0.00, inplace=True)
overall_residential_BP["TotalValuationRemodel"].fillna(0.00, inplace=True)
overall_residential_BP['TotalJobValuation'] = overall_residential_BP['TotalJobValuation'].apply(pd.to_numeric)
overall_residential_BP['TotalValuationRemodel'] = overall_residential_BP['TotalValuationRemodel'].apply(pd.to_numeric)

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

# checkpoint

In [7]:
residential_BP = overall_residential_BP
residential_BP['Issuedmonth'] = pd.DatetimeIndex(residential_BP['IssuedDate']).month

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
residential_BP['OverallStatus'] = np.where(residential_BP['StatusCurrent'] == "Final", "Complete", np.where(residential_BP['StatusCurrent'] == "Active", "Current", "Invalid"))
residential_BP['New_Remodel'] = np.where(residential_BP['WorkClass'] == "New", "New","Remodel_Repair")

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


# checkpoint 2

In [9]:
residential_condensed = residential_BP[['PermitType', 'PermitTypeDesc', 'PermitNum', 'PermitClassMapped',
       'PermitClass', 'WorkClass', 'ProjectName', 'Description',
       'TCAD_ID', 'PropertyLegalDescription', 'IssuedDate',
       'CalendarYearIssued', 'StatusCurrent', 'StatusDate',
       'ExpiresDate', 'CompletedDate', 'TotalExistingBldgSQFT',
       'RemodelRepairSQFT', 'TotalNewAddSQFT', 'TotalValuationRemodel',
       'TotalJobValuation', 'NumberOfFloors', 'HousingUnits',
       'OriginalAddress1', 'OriginalCity', 'OriginalState', 'OriginalZip', 'Location','Issuedmonth',
       'OverallStatus','New_Remodel']]

In [10]:
residential_condensed_valid = residential_condensed.loc[residential_condensed['OverallStatus'] != "Invalid",:]

In [11]:
new_df = residential_condensed_valid.loc[residential_condensed_valid["New_Remodel"] == "New",:]
remodel_df = residential_condensed_valid.loc[residential_condensed_valid["New_Remodel"] != "New",:]

In [12]:
new_df["TotalJobValuation"] = np.where(new_df["TotalJobValuation"]>0,new_df["TotalJobValuation"],new_df["TotalValuationRemodel"])
new_df["TotalValuationRemodel"] = 0

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
remodel_df["TotalValuationRemodel"]= np.where(remodel_df["TotalValuationRemodel"] == 0,remodel_df["TotalJobValuation"],remodel_df["TotalValuationRemodel"])
remodel_df["TotalValuationRemodel"]= np.where(remodel_df["TotalJobValuation"] >= remodel_df["TotalValuationRemodel"], remodel_df["TotalJobValuation"], remodel_df["TotalValuationRemodel"])
remodel_df["TotalJobValuation"] = 0

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [14]:
residential_condensed_valid2 = new_df.append(remodel_df, ignore_index=True)

# checkpoint 3

In [15]:
aggregate_table = pd.pivot_table(residential_condensed_valid2,index=["OriginalZip","CalendarYearIssued","Issuedmonth"],
               values=["PermitType","TotalJobValuation","TotalValuationRemodel"],columns=["New_Remodel"],aggfunc={"PermitType":'count',"TotalJobValuation":np.sum,"TotalValuationRemodel":np.sum},fill_value=0)

In [16]:
aggregate_table = aggregate_table.reset_index()

In [17]:
final_df = aggregate_table[['OriginalZip','CalendarYearIssued','Issuedmonth']]
final_df["NewConstructionsCount"] = aggregate_table["PermitType"]["New"]
final_df["RemodelRepairsCount"] = aggregate_table["PermitType"]["Remodel_Repair"]
final_df["NewConstructionDollarValue"] = aggregate_table["TotalJobValuation"]["New"]
final_df["RemodelRepairDollarValue"] = aggregate_table["TotalValuationRemodel"]["Remodel_Repair"]
final_df.columns = final_df.columns.droplevel(1)
final_df["OriginalZip"] = final_df["OriginalZip"].astype(int).astype(str)
final_df.sort_values(['OriginalZip', 'CalendarYearIssued','Issuedmonth'])

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,OriginalZip,CalendarYearIssued,Issuedmonth,NewConstructionsCount,RemodelRepairsCount,NewConstructionDollarValue,RemodelRepairDollarValue
0,78613,2017,6,0,2,0,10000
1,78617,1996,2,26,0,1593190,0
2,78617,1996,3,6,0,369000,0
3,78617,1996,4,4,1,240000,500
4,78617,1996,5,9,2,613800,1000
5,78617,1996,6,5,0,356000,0
6,78617,1996,7,13,0,934000,0
7,78617,1996,8,7,0,492000,0
8,78617,1996,9,1,0,70000,0
9,78617,1996,10,3,0,210000,0


In [18]:
final_df.to_csv("Construction_Demographics.csv")